In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [16]:
# Load the main dataset
TEST_DF = pd.read_csv("classified_test_df_Hermes-2.csv", index_col=0)

In [17]:
TEST_DF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1884 entries, 0 to 1883
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   post_id                               1884 non-null   object
 1   tweet_text                            1884 non-null   object
 2   key_features                          1884 non-null   object
 3   target                                1615 non-null   object
 4   label                                 1884 non-null   object
 5   annotator_1_label                     1884 non-null   object
 6   annotator_1_target                    1316 non-null   object
 7   annotator_2_label                     1884 non-null   object
 8   annotator_2_target                    1298 non-null   object
 9   annotator_3_label                     1884 non-null   object
 10  annotator_3_target                    1302 non-null   object
 11  Hermes-3-Llama-3.1-70B-Q5_K_S      

In [21]:
TEST_DF['RAG_Hermes-2-70B-i1-IQ2_XXS:latest'].value_counts()

RAG_Hermes-2-70B-i1-IQ2_XXS:latest
offensive                                                                                                                                                                                                                                                         968
hatespeech                                                                                                                                                                                                                                                        770
normal                                                                                                                                                                                                                                                            114
# Only Reply with one word [hate speech, normal, or offensive]\n\nThe text is classified as hate speech due to the following points:\n\n1                                          

In [19]:
import pandas as pd

# Normalization mapping
normalization_map = {
    "offensive": "offensive",
    "Offensive": "offensive",
    "[offensive]": "offensive",
    "[Offensive]": "offensive",
    
    "hate speech": "hatespeech",
    "Hatespeech": "hatespeech",
    "hatespeech": "hatespeech",
    "Hate Speech": "hatespeech",
    "Hate speech": "hatespeech",
    "Hate speech.": "hatespeech",
    "hate speech.": "hatespeech",
    "hatedspeech": "hatespeech",
    "hatemspeech": "hatespeech",
    "haterspeech": "hatespeech",
    "[hate speech]": "hatespeech",
    "[Hate Speech]": "hatespeech",
    
    "normal": "normal",
    "Normal": "normal",
    "Normal.": "normal",
    "[normal]": "normal",
    "[Normal]": "normal",
    
    # Handle cases where the label is inside a list-like structure
    "[Userinput, \"normal\", 1]": "normal",
    "[Userinput, \"offensive\", 1]": "offensive",
    "[Userinput, \"hatespeech\", 1]": "hatespeech",
}


# List of columns to normalize
columns_to_normalize = [
    # "label",
    "Hermes-3-Llama-3.1-70B-Q5_K_S",
    "llama3.3:70B-Instruct-Q2_K",
    "llama3.3:70b-instruct-q6_K-SEEN_DATA",
    "llama3.3:70b-instruct-q6_K",
    "mistral:7b-instruct-v0.2-q8_0",
    "RAG_Hermes-2-70B-i1-IQ2_XXS:latest"
]

# Function to clean and normalize labels
def clean_label(text):
    if pd.isna(text):  # Handle NaN values
        return text
    text = str(text).split(".")[0]  # Keep only text before the first period
    text = text.strip('"').strip()  # Remove quotes and whitespace
    return normalization_map.get(text, text)  # Normalize if in the map

# Apply normalization
for column in columns_to_normalize:
    TEST_DF[column] = TEST_DF[column].astype(str).apply(clean_label)


In [20]:
TEST_DF

,post_id,tweet_text,key_features,target,label,annotator_1_label,annotator_1_target,annotator_2_label,annotator_2_target,annotator_3_label,annotator_3_target,Hermes-3-Llama-3.1-70B-Q5_K_S,llama3.3:70B-Instruct-Q2_K,llama3.3:70b-instruct-q6_K-SEEN_DATA,llama3.3:70b-instruct-q6_K,mistral:7b-instruct-v0.2-q8_0,X_train,label_encoded,RAG_Hermes-2-70B-i1-IQ2_XXS:latest
0,11457099_gab,no means no to illegal aliens <number> buildth...,"['illegal', 'aliens', 'buildthewall', 'kill', ...","Arab, Hispanic, Islam, Refugee",hatespeech,offensive,"Islam, Refugee",hatespeech,"Arab, Hispanic, Islam, Refugee",hatespeech,Refugee,hatespeech,hatespeech,hatespeech,hatespeech,offensive,[ 6.2125096 -0.68465716 -6.3372498 ... 10.51...,0,offensive
1,11551744_gab,not long till the first incident of black mob ...,[],"African, Men, None, Women",hatespeech,hatespeech,"African, Men, Women",offensive,NaN,normal,NaN,hatespeech,hatespeech,hatespeech,hatespeech,offensive,[ 0.7899033 1.2117541 -4.710365 ... 6.32203...,0,offensive
2,1118523950663909377_twitter,<user> dudley is a fucking twat and a pestilen...,[],"Men, Other, Women",hatespeech,hatespeech,"Men, Women, Other",normal,Other,offensive,Other,hatespeech,hatespeech,hatespeech,offensive,offensive,[ 0.08608349 0.26262066 -6.3408113 ... 10.52...,0,offensive
3,27457937_gab,chelsea just piss on me handler just another z...,"['zhid', 'oxygen', 'abuser']","Islam, Jewish",hatespeech,hatespeech,Jewish,hatespeech,Jewish,hatespeech,Islam,hatespeech,hatespeech,hatespeech,hatespeech,offensive,[-0.6398999 0.87283546 0.4512122 ... 11.27...,0,hatespeech
4,1094666527188631552_twitter,<user> we haven ’ t had a president who hated ...,[],"None, Refugee",hatespeech,hatespeech,Refugee,normal,NaN,offensive,Refugee,normal,offensive,offensive,offensive,normal,[ 1.7874744 -0.0422212 -3.9652562 ... 6.74...,0,hatespeech
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879,1164745213648261120_twitter,you don ’ t never fuck yo niggas bitch n you d...,"['fuck', 'yo', 'niggas', 'bitch']","African, Other, Women",offensive,offensive,"African, Women",normal,Other,offensive,"African, Women",hatespeech,hatespeech,hatespeech,hatespeech,offensive,[ 1.3682098 4.4005337 2.4642673 ... 8.22857...,2,hatespeech
1880,1163989090330730498_twitter,<user> eggzactamundo negro im just saying im l...,['negro'],African,offensive,offensive,African,normal,African,offensive,African,offensive,hatespeech,hatespeech,offensive,offensive,[-0.6184644 0.36710113 -1.5807176 ... 5.76...,2,offensive
1881,1125478379304751106_twitter,hm term limits jews worst enemy ‘ tis why flor...,"['term', 'limits', 'jews', 'worst', 'enemy']",Jewish,offensive,offensive,Jewish,offensive,Jewish,hatespeech,Jewish,hatespeech,hatespeech,hatespeech,hatespeech,normal,[ 3.8049612 2.09012 1.3787781 ... 6.64...,2,offensive
1882,11200659_gab,i am sure you are totally the alpha faggot at ...,"['alpha', 'faggot', 'at', 'the', 'standard', '...","Homosexual, Other",offensive,offensive,Other,hatespeech,Homosexual,offensive,Homosexual,hatespeech,hatespeech,hatespeech,hatespeech,offensive,[ 1.3526949 0.62941974 -6.9584336 ... 11.17...,2,hatespeech


In [11]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Define true labels (y_test) and predicted labels (y_pred)
y_test = TEST_DF["label"]
y_pred = TEST_DF["RAG_Mistral:7b-instruct-v0.2-q8_0"]

# Encode labels as integers for evaluation
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)
y_pred_encoded = label_encoder.transform(y_pred)  # Ensure same mapping

# Calculate accuracy
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print(f"Accuracy: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test_encoded, y_pred_encoded, target_names=label_encoder.classes_))


ValueError: y contains previously unseen labels: 'offensive\n\nExplanation: The text implies a hostile and aggressive tone towards the person who unsent a message, but it does not directly target a specific group or individual based on their race, religion, gender, sexual orientation, or other protected characteristic'

In [12]:
main_df = TEST_DF

In [13]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Ensure both columns are cleaned and contain only valid labels
valid_labels = set(main_df["label"].unique())  # Get all expected labels

def clean_prediction(text):
    """Keep only text before the first period and ensure it's a valid label."""
    if pd.isna(text):  # Handle NaN values
        return "unknown"  # Use a placeholder for invalid values
    text = str(text).split(".")[0].strip()
    return text if text in valid_labels else "unknown"  # Keep only valid labels

# Clean the predicted labels
main_df["RAG_Mistral:7b-instruct-v0.2-q8_0"] = main_df["RAG_Mistral:7b-instruct-v0.2-q8_0"].apply(clean_prediction)

# Define true and predicted labels
y_test = main_df["label"]
y_pred = main_df["RAG_Mistral:7b-instruct-v0.2-q8_0"]

# Encode labels
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)
y_pred_encoded = label_encoder.transform(y_pred)  # Now, all labels are in the known set

# Calculate accuracy
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print(f"Accuracy: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test_encoded, y_pred_encoded, target_names=label_encoder.classes_))


ValueError: y contains previously unseen labels: 'unknown'

In [ ]:
TEST_DF['RAG_Hermes-2-70B-i1-IQ2_XXS:latest'].value_counts()

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Extract true labels and predictions
y_test = TEST_DF["label"]
y_pred = TEST_DF["RAG_Hermes-2-70B-i1-IQ2_XXS:latest"]

# Encode labels using LabelEncoder, fitting only on known labels
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)  # Fit on true labels

# Handle unseen labels in y_pred
y_pred_encoded = []
for label in y_pred:
    if label in label_encoder.classes_:
        y_pred_encoded.append(label_encoder.transform([label])[0])  # Encode known labels
    else:
        y_pred_encoded.append(-1)  # Mark unseen labels as -1

y_pred_encoded = np.array(y_pred_encoded)

# Remove invalid predictions (-1)
valid_indices = (y_pred_encoded != -1)
y_test_valid = y_test_encoded[valid_indices]
y_pred_valid = y_pred_encoded[valid_indices]

# Compute accuracy
accuracy = accuracy_score(y_test_valid, y_pred_valid)
print(f"Accuracy: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test_valid, y_pred_valid, target_names=label_encoder.classes_))


Accuracy: 0.4357

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.49      0.62      0.55       611
      normal       0.66      0.12      0.20       619
   offensive       0.37      0.57      0.45       622

    accuracy                           0.44      1852
   macro avg       0.50      0.44      0.40      1852
weighted avg       0.50      0.44      0.40      1852

